In [3]:
from tahoe import execute_async
import pandas as pd
import dvc.api

# taxonomy

In [4]:
df_tax = pd.read_json(dvc.api.get_url( 
    'datasets/data/taxonomy/wish_newtax_02222023.json', 
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

tax2path = {}
for i in df_tax.to_dict('records'):
    tax2path[str(i['id'])] = i['category_path']

# v3 data download

In [36]:
df_v0 = pd.read_json('query_classify_v0_20230307.json', lines=True)

In [37]:
len(df_v0)

1479410

In [6]:
q = f"select query, categories, category_names, weights from sweeper_dev.query_top_10_prediction_v3 where model_version = 3"
res = execute_async(q)

In [7]:
df_v3 = pd.DataFrame(res, columns=['query', 'categories', 'category_names', 'weights'])

In [39]:
len(df_v3)

6627925

In [9]:
# df_v3.to_json('query_classify_v3_20230315.json', lines=True, orient='records')

# load v0v3 data

In [5]:
df_v0 = pd.read_json('query_classify_v0_20230307.json', lines=True)
df_v3 = pd.read_json('query_classify_v3_20230315.json', lines=True)

# top sheet

In [40]:
dfs = pd.read_excel('top_searched_queries_stats.xlsx', sheet_name=None)

In [41]:
df_top = dfs['1000 top searched normalized qu']
df_top.columns = df_top.iloc[0]
df_top = df_top.drop(df_top.index[0])
df_top['query'] = df_top['normalized_query']

In [42]:
df_mid = dfs['1000 Torso searched normalized ']
df_mid.columns = df_mid.iloc[0]
df_mid = df_mid.drop(df_mid.index[0])
df_mid['query'] = df_mid['normalized_query']

df_tail = dfs['1000 tail searched normalized q']
df_tail.columns = df_tail.iloc[0]
df_tail = df_tail.drop(df_tail.index[0])
df_tail['query'] = df_tail['normalized_query']

In [43]:
df_top_merge = df_top.merge(df_v0.rename(columns={'norm': 'v0_norm', 'categories': 'v0_categories', 'category_names': 'v0_category_names', 
    'weights': 'v0_weights'
}), on='query', how='left')

In [44]:
df_mid_merge = df_mid.merge(df_v0.rename(columns={'norm': 'v0_norm', 'categories': 'v0_categories', 'category_names': 'v0_category_names', 
    'weights': 'v0_weights'
}), on='query', how='left')

df_tail_merge = df_tail.merge(df_v0.rename(columns={'norm': 'v0_norm', 'categories': 'v0_categories', 'category_names': 'v0_category_names', 
    'weights': 'v0_weights'
}), on='query', how='left')

In [45]:
df_top_merge = df_top_merge.merge(df_v3.rename(columns={'categories': 'v3_categories', 'category_names': 'v3_category_names', 
    'weights': 'v3_weights'
}), on='query', how='left')

In [46]:
df_mid_merge = df_mid_merge.merge(df_v3.rename(columns={'categories': 'v3_categories', 'category_names': 'v3_category_names', 
    'weights': 'v3_weights'
}), on='query', how='left')

df_tail_merge = df_tail_merge.merge(df_v3.rename(columns={'categories': 'v3_categories', 'category_names': 'v3_category_names', 
    'weights': 'v3_weights'
}), on='query', how='left')

In [47]:
df_top_merge['query_type'] = 'head'
df_mid_merge['query_type'] = 'torso'
df_tail_merge['query_type'] = 'tail'
df_all_merge = pd.concat([df_top_merge, df_mid_merge, df_tail_merge])

In [48]:
df_all_merge[df_all_merge.normalized_query == 'door']

,normalized_query,impression_cnt,clicks_cnt,a2c_cnt,orders_cnt,gmv_sum,%impression,%clicks,%a2c,%orders,...,% unique searches,query,v0_norm,v0_categories,v0_category_names,v0_weights,v3_categories,v3_category_names,v3_weights,query_type
297,door,333798.0,4362.0,518.0,109.0,1026.082649,0.000305,0.000255,0.000223,0.000236,...,0.00038,door,10.86278,"3654,2732,2786,3391,3548","Sealing Strips,Space Separation Tools,Carpets,...","1.52542043408,0.377492846163,0.315293584622,0....","3461,3457,3587,3590","Door & Window Screens,Doors, Gates & Windows,D...","0.2063259631,0.1043876782,0.08764750510000001,...",head


In [49]:
df_all_merge['v0_category_paths'] = df_all_merge['v0_categories'].apply(lambda x: [tax2path[i] if i in tax2path else '' for i in x.split(',')] if isinstance(x, str) else x)
df_all_merge['v3_category_paths'] = df_all_merge['v3_categories'].apply(lambda x: [tax2path[i] if i in tax2path else '' for i in x.split(',')] if isinstance(x, str) else x)

In [50]:
df_all_merge.to_csv('top_searched_queries_stats_with_v0v3.csv', index=False)

# ndcg sheet

In [19]:
df_ndcgs = pd.read_excel('control_vs_show-v1_queries_ndcg_difference_tables_v3.xlsx', sheet_name=None)

In [20]:
tmp = []
for i in df_ndcgs:
    df_i = df_ndcgs[i]
    df_i['query_type'] = i
    tmp.append(df_i)
df_ndcgs = pd.concat(tmp).reset_index()
del df_ndcgs['index']

In [22]:
df_ndcgs['query'] = df_ndcgs['country_gender_query'].apply(lambda x: '_'.join(x.split('_')[2:]))

In [24]:
df_ndcgs_merge = df_ndcgs.merge(df_v0.rename(columns={'norm': 'v0_norm', 'categories': 'v0_categories', 'category_names': 'v0_category_names', 
    'weights': 'v0_weights'
}), on='query', how='left').merge(df_v3.rename(columns={'categories': 'v3_categories', 'category_names': 'v3_category_names', 
    'weights': 'v3_weights'
}), on='query', how='left')

In [25]:
df_ndcgs_merge['v0_category_paths'] = df_ndcgs_merge['v0_categories'].apply(lambda x: [tax2path[i] if i in tax2path else '' for i in x.split(',')] if isinstance(x, str) else x)
df_ndcgs_merge['v3_category_paths'] = df_ndcgs_merge['v3_categories'].apply(lambda x: [tax2path[i] if i in tax2path else '' for i in x.split(',')] if isinstance(x, str) else x)

In [27]:
df_ndcgs_merge.to_csv('control_vs_show-v1_queries_ndcg_difference_tables_with_v0v3.csv', index=False)